<a href="https://colab.research.google.com/github/Ahmed-A-Salem/SuperResolution-GANs/blob/main/SR_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import tensorflow as tf
import numpy as np
import PIL
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import add,Dense
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation,Flatten
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from IPython.display import display
from skimage.transform import rescale, resize
# import georasters as gr
from matplotlib import image
from matplotlib import colors
import glob
from tqdm import tqdm
import imageio
# import rasterio
import argparse
# from rasterio.plot import reshape_as_image
from skimage import img_as_ubyte
import random
from PIL import Image
import cv2

np.random.seed(10)
tf.config.experimental_run_functions_eagerly(True)

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


# Preprocessing

## Functions

### Cropping

In [ ]:
# #create list of crops with coordinates Top-Left
# def select_crops(initial_width,initial_height,target_size=384):
#     crop_rect = []
#     initial_width = initial_width - target_size
#     initial_height = initial_height - target_size
#     for x in range(0,initial_width,target_size):
#         for y in range(0,initial_height,target_size):
#             crop_rect.append({"left": x, "top": y})
#     print("total number of crops= ", len(crop_rect))
#     return crop_rect


### Convert Gtiff to png

In [ ]:
# def convert_s2(input_path, output_path, crop_rect, target_size, bands, rgb_mode=False,s2_file_name=None):
#     with rasterio.open(input_path) as s2_data:
#         s2 = s2_data.read(bands)
#     cat_image = reshape_as_image(s2)

#     if s2_file_name is None :
#         file_name = input_path
#     else:
#         file_name = s2_file_name
#     file_name = file_name.split("/")[-1].split(".")[0]
#     file_name = os.path.join(output_path, file_name)

#     for index in range(len(crop_rect)):
#         left = crop_rect[index]["left"]
#         top = crop_rect[index]["top"]
#         right = left + target_size
#         bottom = top + target_size
#         new_image = cat_image[top:bottom, left:right]
#         new_image = np.moveaxis(new_image, -1, 0)
#         new_file_name = file_name + "_" + str(index) + ".tif"
#         if rgb_mode:
#             display_channels = np.array(bands) - 2
#             display_channels = display_channels.tolist()
#             s2 = new_image.astype(np.float32)
#             min_value, max_value = np.min(s2), np.max(s2)
#             s2 = np.clip(s2, min_value, max_value)
#             s2 /= max_value
#             s2 = s2.astype(np.float32)
#             #s2 = s2[display_channels, :, :]
#             rgb = np.rollaxis(s2, 0, 3)
#             imageio.imsave(new_file_name.replace('tif', 'png'), img_as_ubyte(rgb))

#         else:
#             with rasterio.open(new_file_name, 'w',width=target_size, height=target_size,
#                                count=len(bands),driver="Gtiff", dtype=rasterio.uint16) as dst:
#                 dst.write(new_image)


In [ ]:
# def cut_sen2_files(target_size=384):
#     path = '/data/super_resolution/original/'       # path of original hr images
#     output_path = '/data/super_resolution/output'   #save high resolution data after cropping here
#     for p in os.listdir(path):
#         f= gr.from_file(path+p)
#         file_name = p
#         ch,initial_height, initial_width = f.shape
#         crop_rect = select_crops(initial_width, initial_height, target_size)
#         bands = [5, 3, 2]
#         convert_s2(path+file_name, output_path ,crop_rect, target_size, bands, rgb_mode=True)


## Calling Preprocessing Functions

In [ ]:
# cut_sen2_files()

### Denoise & best Contour

In [ ]:
# path_hr = '/content/outputhr/'
# path_hr_processed = '/content/outputhr_processed/'
# def denoise_contour():
#   for p in os.listdir(path_hr):
#     img = cv2.imread(path_hr+p)
#     noiseless_image_colored = cv2.fastNlMeansDenoisingColored(img,None,20,20,7,21)
#     gray = cv2.cvtColor(noiseless_image_colored,cv2.COLOR_BGR2GRAY)
#     _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
#     contours,hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     cnt = contours[0]
#     x,y,w,h = cv2.boundingRect(cnt)
#     crop = noiseless_image_colored[y:y+h,x:x+w]
#     img = Image.fromarray(crop)
#     img.save(path_hr_processed+p, 'png')

In [ ]:
# lr_shape=(96, 96, 3)
# #path of cropped hr images
# path = '/data/super_resolution/output/'
# # path that you need to save lr images in
# outputpath = '/data/super_resolution/input/'
# for f in os.listdir(path):
#     image = Image.open(path+f) 
#     lr_resized=np.array(image.resize((lr_shape[0],lr_shape[1]),PIL.Image.BICUBIC))
#     img = Image.fromarray(lr_resized)
#     img.save(outputpath+f, 'png')

In [ ]:
def load_images_from_folder(folder): 
    images = [] 
    for filename in os.listdir(folder): 
        if (filename == '.ipynb_checkpoints'):
          continue
        img = Image.open(os.path.join(folder,filename)) 
        img_resized=np.array(img)
        images.append(img_resized) 
    return images

output= load_images_from_folder('/content/hr')
input= load_images_from_folder('/content/lr')
len(input), len(output)

(2, 2)

In [ ]:
np.shape(output)

(2, 384, 384, 3)

In [ ]:
input_ = np.array(input)
output_ = np.array(output)

In [ ]:
def normalize_img(input_data):
    return (input_data.astype(np.float32) - 127.5)/127.5 

def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8)

In [ ]:
input = normalize_img(input_)
output = normalize_img(output_)
print(len(input)), print(len(output))

2
2


(None, None)

# Training

In [ ]:
X_train_lr, X_test_lr, X_train_hr, X_test_hr = train_test_split(input, output, test_size=0.5, random_state=42)
print(len(X_train_lr), len(X_train_hr))

1 1


## Building Training Functions

In [ ]:
###Upsample function(changing from H * W * C to H * W * 4C then to 2H * 2W * C using pixelshuffling)
def upsample(model,filter_size,no_of_channels,strides):
    #scaling factor=2
    scale=2
    no_of_filters=no_of_channels *(scale ** 2)
    model=Conv2D(filters=no_of_filters,kernel_size=filter_size,strides=strides,padding='same')(model)
    model=UpSampling2D(size=scale)(model)
    model=PReLU()(model)
    return model

In [ ]:
def residual_block(model, kernel_size, no_of_filters, strides):  
    gen = model
    model = Conv2D(filters = no_of_filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    
    # Using Parametric ReLU
    model = PReLU(alpha_initializer='zeros', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(model)
    model = Conv2D(filters = no_of_filters, kernel_size = kernel_size, strides = strides, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    
    model = add([gen, model])
    
    return model

### Generator Network

In [ ]:
# Using Functional API of Keras

def generator_network(gen_input):

    gen_input = Input(shape = gen_input)
     
    model = Conv2D(filters = 64, kernel_size = 9, strides = 1, padding = "same")(gen_input)
    model = PReLU(shared_axes=[1,2])(model)               # each filter only has one set of parameters
    model_part1 = model
        
  # Using 16 Residual Blocks
    for i in range(16):
        model = residual_block(model, 3, 64, 1)
        # 16 residual blocks with skip connections 

    model = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = "same")(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model = add([model_part1, model])                      
  #  Element wise of model_part1 and model after 16 residual blocks
     

    for i in range(2):
        model = upsample(model, 3, 64, 1)  #no of channels=64  
     
    model = Conv2D(filters = 3, kernel_size = 9, strides = 1, padding = "same")(model)
  
    model = Activation('tanh')(model)                  # tanh activation in last layer
    
    gen_model = Model(inputs = gen_input, outputs = model)     # specifying the input and output to the model
  
    return gen_model

### Discriminator Network

In [ ]:
def conv_disc_block(model,filters,filter_size,strides):
    model=Conv2D(filters=filters,kernel_size=filter_size,strides=strides,padding='same')(model)
    model = BatchNormalization(momentum = 0.5)(model)
    model=LeakyReLU(alpha=0.1)(model)
    return model

In [ ]:
def discriminator_network(image_shape):
    disc_input=Input(shape = image_shape)
  #convolution layer(k3n64s1)
    model=Conv2D(filters = 64,kernel_size = 3,strides=1,padding='same' )(disc_input)
  #Activation-leaky relu
    model=LeakyReLU(alpha=0.1)(model)
  
  #discriminator block (k3n64s2)
    model=conv_disc_block(model,64,3,2)
  #discriminator block (k3n128s1)
    model=conv_disc_block(model,128,3,1)
  #discriminator block (k3n128s2)
    model=conv_disc_block(model,128,3,2)
  #discriminator block (k3n256s1)
    model=conv_disc_block(model,256,3,1)
  #discriminator block (k3n256s2)
    model=conv_disc_block(model,256,3,2)
  #discriminator block (k3n512s1)
    model=conv_disc_block(model,512,3,1)
  #discriminator block (k3n512s2)
    model=conv_disc_block(model,512,3,2)

  #for dense layer input should be column vector/flatten
    model=Flatten()(model)
  #dense layer with 1024 nodes
    model=Dense(1024)(model)
  #Activation-leaky relu
    model=LeakyReLU(alpha=0.1)(model)

  #dense layer with 1 nodes
    model=Dense(1)(model)
  #Activation-sigmoid
    model=Activation('sigmoid')(model)
    disc_model=Model(inputs=disc_input,outputs=model)
    return disc_model

### GAN Network

In [ ]:
def gan_network(generator_model,discriminator_model,shape):
    discriminator_model.trainable = False
    gan_input=Input(shape=shape)
    print("input")
    SR=generator_model(gan_input)
    print("SR")
    gan_output=discriminator_model(SR)
    print("gan_output")
    model=Model(inputs=gan_input,outputs=[SR,gan_output])
    return model

### Loss Functions

In [ ]:
# To use mean, square we need to use keras.backend
# For content loss, compare the results which the vgg19 provides which feeding the y_true and y_pred
def content_loss(image_shape):
    def loss( y_true, y_pred):
        vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=image_shape)
        vgg19.trainable = False
        for layer in vgg19.layers:
            layer.trainable = False
        model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
        model.trainable = False
        return K.mean(K.square(model(y_true) - model(y_pred)))
    return loss 

### Plot

In [ ]:
def plot_generated_images(epoch,generator, examples=3 , dim=(1, 3), figsize=(15, 5)):  
    
    rand_nums = np.random.randint(0, X_test_hr.shape[0], size=examples)
    image_batch_hr = denormalize(X_test_hr[rand_nums])
    image_batch_lr = X_test_lr[rand_nums]
    gen_img = generator.predict(image_batch_lr)
    generated_image = denormalize(gen_img)
    image_batch_lr = denormalize(image_batch_lr)
    
    
    plt.figure(figsize=figsize)
    
    plt.subplot(dim[0], dim[1], 1)
    plt.imshow(image_batch_lr[1], interpolation='nearest')
    plt.axis('off')
        
    plt.subplot(dim[0], dim[1], 2)
    plt.imshow(generated_image[1], interpolation='nearest')
    plt.axis('off')
    
    plt.subplot(dim[0], dim[1], 3)
    plt.imshow(image_batch_hr[1], interpolation='nearest')
    plt.axis('off')
    
    plt.tight_layout() 

### Model Train

In [ ]:
from tqdm import tqdm

hr_shape=(384,384,3)
lr_shape=(hr_shape[0]/4,hr_shape[1]/4,hr_shape[2])

def train_model(batch_size,epochs):
    no_of_batches=X_train_hr.shape[0]//batch_size
    adam = Adam(lr=0.0001 ,beta_1=0.9 ,beta_2=0.999, epsilon=1e-08 )
    discriminator_model = discriminator_network(hr_shape)
    generator_model = generator_network(lr_shape)
    generator_model.compile(loss=content_loss(hr_shape), optimizer=adam)
    discriminator_model.compile(loss='binary_crossentropy',optimizer=adam)

    gan_model=gan_network(generator_model,discriminator_model,lr_shape)
    discriminator_model.trainable = False
    gan_model.compile(loss=[content_loss(hr_shape),'binary_crossentropy'],loss_weights=[1.0,1e-3],optimizer=adam)


    for i in range(0,epochs):
        print("\nEpoch    : "+ str(i))

        for j in range(no_of_batches):

            print("Batch    : "+str(j))

            rand_nums = np.random.randint(0, X_train_hr.shape[0], size=batch_size)

            image_batch_hr = X_train_hr[rand_nums]
            image_batch_lr = X_train_lr[rand_nums]

            batch_gen_sr = generator_model.predict(image_batch_lr)
            real_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2      ## Here we use concept of label smoothing
            fake_data_Y = np.random.random_sample(batch_size)*0.2

            discriminator_model.trainable = True
            d_loss_real = discriminator_model.train_on_batch(image_batch_hr, real_data_Y)
            d_loss_fake = discriminator_model.train_on_batch(batch_gen_sr, fake_data_Y)
            d_loss = 0.5*np.add(d_loss_fake, d_loss_real)

            discriminator_model.trainable = False     
            gan_data_Y = np.ones(batch_size) - np.random.random_sample(batch_size)*0.2
            loss_gan = gan_model.train_on_batch(image_batch_lr, [image_batch_hr,gan_data_Y])


        print("discriminator_loss : %f" % d_loss)
        print("gan_loss :", loss_gan)
        loss_gan = str(loss_gan)

        # loss_file = open( '/kaggle/working/losses.txt' , 'a')
        # loss_file.write('epoch%d : gan_loss = %s ; discriminator_loss = %f\n' %(i, loss_gan, d_loss) )
        # loss_file.close()

        plot_generated_images(i, generator_model)

        generator_model.save('/kaggle/working/Super-Resolve/gen_model.h5')
        # discriminator_model.save('/kaggle/working/Super-Resolve/dis_model.h5')

# Run and Results

In [ ]:
lr_shape = tuple(map(int, lr_shape))

In [ ]:
# Not Enough RAM to train on kaggle, but I have successfully trained it on colab
# To train with batch size 4 and epochs 50

train_model(1, 2)